##Required installation






In [ ]:
!pip install transformers -q --quiet
!pip install datasets -q --quiet
!pip install tensorflow -q --quiet

##Importing data and processing the data

In [ ]:
TRAIN_DATASET_PATH = "train.csv" # directory path where the training dataset is located
TEST_DATASET_PATH = "test.csv" # directory path where the test dataset is located

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df = test_df.dropna()
train_df = train_df.dropna()

# This hyperparameter can be set and has a preferred range between 0.1 and 0.2
validation_split = 0.1
train_df,val_df = train_test_split(train_df, test_size = validation_split, random_state = 100)
# When splitting data, an index column will be added. It is dropped using this command
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
from datasets import Dataset
import pyarrow as pa

# Converting DataFrame to a Dataset: a required step for using hugging face pipline
df_train = Dataset(pa.Table.from_pandas(train_df))
df_test = Dataset(pa.Table.from_pandas(test_df))
df_val = Dataset(pa.Table.from_pandas(val_df))
print(df_train,df_test,df_val)

Dataset({
    features: ['news', 'source', 'label'],
    num_rows: 2069
}) Dataset({
    features: ['news', 'source', 'label'],
    num_rows: 500
}) Dataset({
    features: ['news', 'source', 'label'],
    num_rows: 230
})


##Importing Model and ModelTokeniszer

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# This is a Hugging Face model token which is a unique identifier used to instantiate and utilize pre-trained models through the Hugging Face library.
model_tk = "mrm8488/xlm-roberta-base-finetuned-HC3-mix"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = model_tk) # Calling the tokensizer for the model

##Tokenization of the Data

In [ ]:
# Tokenization function
def tokenize(batch):
    return tokenizer(batch['news'], padding = True, truncation = True, add_special_tokens = True, max_length = 512)
# We are tokenizing the the data
df_train_tokenised = df_train.map(tokenize, batched=True)
df_test_tokenised = df_test.map(tokenize, batched=True)
df_val_tokenised = df_val.map(tokenize, batched=True)
print(df_train_tokenised,df_test_tokenised,df_val_tokenised)

Map:   0%|          | 0/2069 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Dataset({
    features: ['news', 'source', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2069
}) Dataset({
    features: ['news', 'source', 'label', 'input_ids', 'attention_mask'],
    num_rows: 500
}) Dataset({
    features: ['news', 'source', 'label', 'input_ids', 'attention_mask'],
    num_rows: 230
})


##Data Reconstruction to accommodate the model

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

# Data collator is a simple object which divides the data in to batches.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")
# Hyperparameter you can vary to change the batch size
batch_size = 16
# Create tensorflow datasets for train and validation
tf_train_dataset = df_train_tokenised.to_tf_dataset(
    columns = ['input_ids'],
    label_cols = ['label'],
    shuffle = False,
    batch_size = batch_size,
    collate_fn = data_collator
)
tf_valid_dataset = df_val_tokenised.to_tf_dataset(
    columns = ['input_ids'],
    label_cols = ['label'],
    shuffle = False,
    batch_size = batch_size,
    collate_fn = data_collator
)
print(tf_train_dataset,tf_valid_dataset)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:385: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))> <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 43), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


##Optimizer and Loss Function

In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay


# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 20
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.0,
    decay_steps=num_train_steps
)

In [ ]:
import tensorflow.keras.optimizers as opt

# The Adam optimizer was imported and utilized in conjunction with the previously defined scheduler.
# This choice was made as the scheduler was found to be effective in enhancing the results by regulating the learning rate.
optimizer = opt.Adam(learning_rate=lr_scheduler,
    weight_decay=0.004,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07)

##Model Compilation and Execution



In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras import metrics
from tensorflow.keras import losses

# Calling the model with its pretrained weights
model = TFAutoModelForSequenceClassification.from_pretrained(model_tk,from_pt=True,trainable=True)
# Compilation of the model with the above definied optimiser and using SparseCategoricalCrossentropy and  SparseCategoricalAccuracy metrics which were
model.compile(
    optimizer = optimizer,
    loss = losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = metrics.SparseCategoricalAccuracy()
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [ ]:
from keras.callbacks import EarlyStopping

# This early stopping callback is  used in order to prevent overfitting
# Here patience is a hyperparameter that can be set
early_stopping_callback = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 2, patience = 6)

In [ ]:
# Fit the model
history_embedding = model.fit(tf_train_dataset, batch_size = batch_size, validation_data = tf_valid_dataset, epochs = num_epochs, callbacks = [early_stopping_callback])

Epoch 1/20
130/130 [==============================] - 97s 349ms/step - loss: 0.7213 - sparse_categorical_accuracy: 0.6535 - val_loss: 0.6635 - val_sparse_categorical_accuracy: 0.6304
Epoch 2/20
130/130 [==============================] - 34s 260ms/step - loss: 0.6473 - sparse_categorical_accuracy: 0.6588 - val_loss: 0.6599 - val_sparse_categorical_accuracy: 0.6304
Epoch 3/20
 76/130 [================>.............] - ETA: 14s - loss: 0.6496 - sparse_categorical_accuracy: 0.6521

##Visualizations and Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the graph between training and  validation loss
plt.plot(history_embedding.history['loss'], c = 'b', label = 'train loss')
plt.plot(history_embedding.history['val_loss'], c = 'r', label = 'validation loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np

# Predicting the labels for the test dataset
outputs = model.predict(df_test_tokenised['input_ids'])
outputs = outputs['logits'].tolist()
# The "argmax" function was employed to determine the index of the maximum value, which serves as our label indirectly.
pred = np.argmax(outputs,axis=1)

In [ ]:
from sklearn.metrics import classification_report

# Classification report
print(classification_report( pred, test_df['label']))

In [ ]:
from sklearn.metrics import  confusion_matrix

# Confusion matrix
cm = confusion_matrix(df_test['label'], pred)
cm_matrix = pd.DataFrame(data = cm, columns = ['Actual Positive: 0', 'Actual Negative: 1'], index = ['Predict Positive: 0', 'Predict Negative: 1'])
sns.heatmap(cm_matrix, annot = True, fmt = 'd', cmap = 'YlGnBu')